## news 데이터 불러오기

In [1]:
import json
import os
from glob import glob

In [2]:
news_dir = '../data/newsdata'
glob(news_dir + '/*')

['../data/newsdata/news_1_1', '../data/newsdata/news_2_0']

In [3]:
news_list = []

for news_dir_i in glob(news_dir + '/*'):
    tmp_news_list = glob(news_dir_i + '/*')
    news_list += tmp_news_list

In [4]:
news_list[:5]

['../data/newsdata/news_1_1/NWRW2000000002.json',
 '../data/newsdata/news_1_1/NLRW2000000019.json',
 '../data/newsdata/news_1_1/NLRW2000000023.json',
 '../data/newsdata/news_1_1/NLRW2000000003.json',
 '../data/newsdata/news_1_1/NLRW2000000015.json']

In [5]:
news_list = [x for x in news_list if x.endswith('.json')]
news_list.sort()

In [6]:
news_list[:5]

['../data/newsdata/news_1_1/NIRW2000000001.json',
 '../data/newsdata/news_1_1/NIRW2000000002.json',
 '../data/newsdata/news_1_1/NLRW2000000001.json',
 '../data/newsdata/news_1_1/NLRW2000000002.json',
 '../data/newsdata/news_1_1/NLRW2000000003.json']

In [7]:
import re
from kss import split_sentences

def preprocess(text):
    # 한 개의 평서문만 허용
    if text.count('.') != 1:
        return ''
    if '·' in text:
        return ''
    # \xa0
    text = re.sub('\xa0', ' ', text)

    # 괄호 안의 텍스트 제거
    text = re.sub(r'\([^)]*\)','', text)
    # <> 안의 텍스트 제거
    text = re.sub('<.+?>', '', text)
    # 큰따옴표 제거
    text = re.sub('"', '', text)
    # 다른 형태의 따옴표 처리
    text = re.sub(r'[‘’]', '', text)
    text = re.sub(r'[“”]', '', text)
    
    bullets = r'[◇▲○■▶◆#※♢]'
    text = re.sub(bullets, '', text)
    
    # 영어, 한국어를 제외한 다른 언어 제외
    if re.sub("[가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9.\W]", '', text):
        return ''
    
    # 너무 적은 개수의 토큰으로 이루어진 경우 제외(예: 어린 시절.)
    if len(text.split()) <= 5:
        return ''
    
    return text      

In [8]:
default_dir = '../data/newsdata/news_1_1/'
sample = default_dir + 'NPRW2000000001.json'

In [9]:
with open(sample, 'r') as f:
    json_data = json.load(f)


In [10]:
json_data.keys()

dict_keys(['id', 'metadata', 'document'])

In [11]:
json_data['id']

'NPRW2000000001'

In [12]:
json_data['metadata']

{'title': '국립국어원 신문 말뭉치 NPRW2000000001',
 'creator': '국립국어원',
 'distributor': '국립국어원',
 'year': '2019',
 'category': '신문>전문지',
 'annotation_level': ['원시'],
 'sampling': '본문 전체'}

In [13]:
len(json_data['document'])

24745

In [14]:
json_data['document'][0]

{'id': 'NPRW2000000001.1',
 'metadata': {'title': '한국경제 2019년 기사',
  'author': '박동휘(donghuip@hankyung.com) 주용석(hohoboy@hankyung.com)',
  'publisher': '한국경제',
  'date': '20190120',
  'topic': '사회',
  'original_topic': '정치,북한|정치,외교|국제,미국_북미'},
 'paragraph': [{'id': 'NPRW2000000001.1.1',
   'form': '\'김정은 친서\' 파격제안 있었나…트럼프 "믿을수 없을 만큼 좋은 만남"'},
  {'id': 'NPRW2000000001.1.2',
   'form': '북핵 협상의 ‘진짜 게임’이 시작됐다. 시한은 2월 말까지다. 도널드 트럼프 미국 대통령은 19일(현지시간) 다음달 말에 2차 미·북 정상회담이 열릴 것이라며 ‘데드라인’을 제시했다. 앞으로 한 달, 우리 정부 중재 속에 미국과 북한 간 한반도의 ‘핵운명’을 좌우할 세기의 협상이 펼쳐진다. ‘김정은 북한 국무위원장 친서’를 소지한 김영철 북한 노동당 부위원장과 트럼프 대통령의 90분 백악관 면담이 교착 상태에 빠져 있던 북핵 협상을 본궤도에 올려놨다는 평가가 나온다.'},
  {'id': 'NPRW2000000001.1.3', 'form': '급박했던 김영철의 2박3일'},
  {'id': 'NPRW2000000001.1.4',
   'form': '트럼프 대통령은 김영철과의 면담 하루 만에 김정은과의 두 번째 만남을 공식화했다. 정상회담 장소도 “우리는 한 나라를 선택했다”고 밝혔다. 지난 17일 김영철의 워싱턴DC 도착 직후에도 이례적인 침묵을 지키던 트럼프 대통령은 18일 90분간의 면담에 대해 “거의 두 시간 동안 만났다”며 “믿을 수 없을 만큼 매우 좋은 만남이었다”고 말했다.'},
  {'id': 'NPRW2000000001.1.5',
   'form': '김영철이 베이징

## news topic 별로 데이터 모으기

In [15]:
topic_dict = {}

for i in range(len(json_data['document'])):
    tmp_topic = json_data['document'][i]['metadata']['topic']
    if tmp_topic not in topic_dict:
        topic_dict[tmp_topic] = [i]
    else:
        topic_dict[tmp_topic].append(i)

In [16]:
topic_dict.keys()

dict_keys(['사회', 'IT/과학', '경제', '연예', '생활', '정치', '미용/건강', '문화', '스포츠'])

In [17]:
for k in topic_dict.keys():
    print(f"{k}: {len(topic_dict[k])}")

사회: 5159
IT/과학: 2368
경제: 6071
연예: 3705
생활: 2975
정치: 1596
미용/건강: 1208
문화: 1019
스포츠: 644


In [18]:
def news_by_topics(news_file):
    topic_dict = {}
    
    with open(news_file, 'r') as f:
        json_data = json.load(f)
    
    for i in range(len(json_data['document'])):
        tmp_topic = json_data['document'][i]['metadata']['topic']
        tmp_data = json_data['document'][i]['paragraph']
        
        if tmp_topic not in topic_dict:
            topic_dict[tmp_topic] =  []
            
        for j in range(len(tmp_data)):
            if j == 0: continue
            if len(tmp_data[j]['form']) <= 70:
                topic_dict[tmp_topic].append(tmp_data[j]['form'])
            
    for top in topic_dict:
        topic_dict[top] = [preprocess(x) for x in topic_dict[top] if preprocess(x)]
    
    return topic_dict

In [19]:
sentence_dict = news_by_topics(sample)

In [20]:
sentence_dict.keys()

dict_keys(['사회', 'IT/과학', '경제', '연예', '생활', '정치', '미용/건강', '문화', '스포츠'])

In [21]:
for k in sentence_dict.keys():
    print(f"{k}: {len(sentence_dict[k])}")

사회: 4317
IT/과학: 1234
경제: 2876
연예: 5655
생활: 2140
정치: 1074
미용/건강: 635
문화: 1001
스포츠: 538


In [22]:
sentence_dict['스포츠'][:10]

["황재균이 '나 혼자 산다'에서 손아섭, 전준우와 우정과 함께 연봉 서열까지 공개했다.",
 '한혜진은 정말 친한 이들만이 할 수 있는 것이라고 세 사람의 우정을 치켜 세웠다.',
 "이날 방송에서도 손아섭에 대해 'KBO 연봉 TOP5의 위엄'이라면서 손아섭의 연봉을 전했다.",
 '한편 올해 첫 세계랭킹 순위 변동은 오는 3일 열리는 센트리 토너먼트 오브 챔피언스에서 판가름이 날 전망이다.',
 '손흥민이 잉글랜드 프로축구 프리미어리그 득점과 도움 부문에서 톱10에 진입했다.',
 '키움과 히어로즈는 유사한 유전자를 갖고 있습니다.',
 '골프 안 했으면요? 음, 아마 특전사에 들어갔거나 학군장교가 됐을걸요? 하하하.',
 '손으로만 속도를 내려고 하니까 오히려 거리가 안 나는 거예요.',
 '특히, 양질의 일자리 창출을 통한 지역 경제 활성화를 위해 투자유치와 도민회를 중심으로 한 일본 관광객 유치 협조를 당부했다.',
 '이마히라가 초청선수로 결정됨에 따라 올해 마스터스 출전 선수는 총 80명으로 늘었다.']

In [23]:
sentence_dict['IT/과학'][:10]

['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
 '삼성전자 관계자는 지난해부터 중저가 모델에 멀티 카메라 등 프리미엄 기능을 먼저 넣고 있다고 말했다.',
 '국대폰에서 출고가 기준 최대 100% 할인 이벤트를 진행하면서 스마트폰을 저렴하게 구매하기 위한 소비자들의 발길을 잡고 있다.',
 'A시리즈 하위 모델은 후면 듀얼 카메라가, 상위 모델에는 후면 트리플 혹은 쿼드 카메라가 달릴 것으로 점쳐진다.',
 '지난해부터 중저가 모델에 멀티카메라 등 프리미엄 기능을 적극 탑재한 삼성전자의 기조가 올해도 이어질 것으로 보인다.',
 '5G 스마트폰 출시를 앞두고 소비자들이 배터리 소모에 대한 우려를 드러냈다.',
 'SK그룹이 전기차 배터리와 자율주행 기술, 차량용 반도체, 고부가가치 소재 사업을 통해 미래자동차 시장 선점에 나선다.',
 '대학원생들이 교수를 평가하는 인터넷 사이트가 있다던데….',
 '5세대 이동통신 시대 1등 사업자가 되겠다.',
 '택시업계 관계자는 이달 안에 티원택시 앱을 일반인들에게 선보일 것으로 예상된다고 말했다.']

In [24]:
sentence_dict['연예'][:10]

['SBS 신개념 음악 예능 더 팬이 단 한 회도 빠짐없이 2049 타깃 시청률 1위를 기록하고 있다.',
 '한편 최종 TOP5는 현장 관객 점수와 앱 투표 결과를 합산해 결정되며, 그 결과는 다음주 생방송 무대에서 공개된다.',
 '동물의 사생활 문근영 김혜성 에릭남이 촬영 열정을 불태웠다.',
 '한편 동물의 사생활은 매주 금요일 오후 8시 55분 방송된다.',
 "유아인이 '도올아인 오방간다'를 통해 성차별에 대한 견해를 밝혔다.",
 'JTBC 금토극 SKY 캐슬이 역대 비지상파 프로그램 최고 시청률을 기록했다.',
 '하하와 별이 서로에 대한 애정을 드러냈다.',
 "'봄이 오나 봄' 이유리와 엄지원의 화기애애한 촬영 현장 모습이 포착됐다.",
 "전혀 다른 생활을 하던 두 사람이 갑자기 몸을 바뀌게 되면서 펼쳐지는 이야기가 '봄이 오나 봄'의 주요 줄거리다.",
 '봄이 오나 봄 이유리와 엄지원의 화기애애한 촬영현장 모습이 포착됐다.']

## news classification 데이터셋 만들기

In [25]:
import pandas as pd

df = pd.DataFrame(columns=['sentence', 'topic'])

In [26]:
sent_topic = []

for top in sentence_dict:
    for sent in sentence_dict[top]:
        sent_topic.append([sent, top])
        

In [27]:
len(sent_topic)

19470

In [28]:
df = pd.DataFrame(sent_topic, columns=['sentence', 'topic'])

In [29]:
df.head()

,sentence,topic
0,2차 북미정상회담이 열릴 나라도 정해졌지만 추후 발표할 것이라고 덧붙였다.,사회
1,현대백화점그룹 계열 종합식품 기업인 현대그린푸드가 국내 중소 식품회사의 가정간편식을...,사회
2,"미국의 상응 조치로는 평양 연락사무소 개설을 비롯해 종전선언, 개성공단 및 금강산관...",사회
3,강성태가 인기 드라마 'SKY캐슬' 속 입시 코디네이터의 존재에 대해 언급했다.,사회
4,강성태는 지난 15일 자신의 유튜브 채널을 통해 정치인이나 기업인 분들에게 코디 제...,사회


In [30]:
import random

df = df.sample(frac=1)

In [31]:
df.head()

,sentence,topic
1012,도는 올해 처음으로 소규모 노후산단을 개선하는 YES산단 육성사업을 도입해 지속 지...,사회
7578,대출 금리도 인하 요인을 늘려 중신용자들의 금리부담 완하에 앞장선다.,경제
2485,"각각 627명, 705명이 필기시험에 응시해 90명과 120명이 이후 면접전형 기회...",사회
12719,"한편 99억의 여자는 매주 수, 목 오후 10시에 방송된다.",연예
10805,한편 JTBC 월화드라마 열여덟의 순간 2회는 23일 밤 9시 30분에 방송된다.,연예


In [32]:
df.reset_index(drop=True, inplace=True)

df.head()

,sentence,topic
0,도는 올해 처음으로 소규모 노후산단을 개선하는 YES산단 육성사업을 도입해 지속 지...,사회
1,대출 금리도 인하 요인을 늘려 중신용자들의 금리부담 완하에 앞장선다.,경제
2,"각각 627명, 705명이 필기시험에 응시해 90명과 120명이 이후 면접전형 기회...",사회
3,"한편 99억의 여자는 매주 수, 목 오후 10시에 방송된다.",연예
4,한편 JTBC 월화드라마 열여덟의 순간 2회는 23일 밤 9시 30분에 방송된다.,연예


In [33]:
df['topic'].value_counts()

연예       5655
사회       4317
경제       2876
생활       2140
IT/과학    1234
정치       1074
문화       1001
미용/건강     635
스포츠       538
Name: topic, dtype: int64

## label 인코딩

In [34]:
topic_labels = df['topic'].unique()

topic_labels

array(['사회', '경제', '연예', '생활', '문화', 'IT/과학', '정치', '미용/건강', '스포츠'],
      dtype=object)

In [35]:
label_dict = {}

for i, topic in enumerate(topic_labels):
    label_dict[topic] = i

label_dict

{'사회': 0,
 '경제': 1,
 '연예': 2,
 '생활': 3,
 '문화': 4,
 'IT/과학': 5,
 '정치': 6,
 '미용/건강': 7,
 '스포츠': 8}

In [36]:
df['label'] = df['topic'].replace(label_dict)

df.head()

,sentence,topic,label
0,도는 올해 처음으로 소규모 노후산단을 개선하는 YES산단 육성사업을 도입해 지속 지...,사회,0
1,대출 금리도 인하 요인을 늘려 중신용자들의 금리부담 완하에 앞장선다.,경제,1
2,"각각 627명, 705명이 필기시험에 응시해 90명과 120명이 이후 면접전형 기회...",사회,0
3,"한편 99억의 여자는 매주 수, 목 오후 10시에 방송된다.",연예,2
4,한편 JTBC 월화드라마 열여덟의 순간 2회는 23일 밤 9시 30분에 방송된다.,연예,2


## data split

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                 df.label.values,
                                                 test_size=0.2,
                                                 random_state=42,
                                                 stratify=df.label.values)


In [38]:
X_train[:5]

array([ 9722, 12895, 16961, 14568, 13496])

In [39]:
y_train[:5]

array([5, 5, 0, 0, 1])

In [40]:
X_val[:5]

array([ 6222, 12311,  9557, 15277,  9811])

In [41]:
y_val[:5]

array([2, 1, 1, 1, 2])

In [42]:
df['data_type'] = ['not_set'] * df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['topic', 'label', 'data_type']).count()

sentence
topic label data_type          
IT/과학 5     train           987
            val             247
경제    1     train          2301
            val             575
문화    4     train           801
            val             200
미용/건강 7     train           508
            val             127
사회    0     train          3454
            val             863
생활    3     train          1712
            val             428
스포츠   8     train           430
            val             108
연예    2     train          4524
            val            1131
정치    6     train           859
            val             215

In [43]:
df.iloc[X_train].head()

,sentence,topic,label,data_type
9722,유전체 기반 오픈 플랫폼 마이지놈박스가 공식 웹사이트 오픈 4주년인 오는 8월 15...,IT/과학,5,train
12895,해당 채널은 조 장관 딸 조모씨에 대한 영상을 올리며 실명과 얼굴을 그대로 노출하며...,IT/과학,5,train
16961,뉴욕 증시는 그동안 무역합의가 이뤄질 것이란 데 대해 매우 낙관적이었다.,사회,0,train
14568,이어 금번 WFP를 통해 지원되는 식량이 북한 주민에게 최대한 신속히 전달될 수 있...,사회,0,train
13496,한국은행 통합별관 건축을 결국 계룡건설이 맡게 될 것으로 보인다.,경제,1,train


In [44]:
df.iloc[X_val].head()

,sentence,topic,label,data_type
6222,한편 캠핑클럽은 매주 일요일 오후 9시에 방송된다.,연예,2,val
12311,장 대표는 시예저치앙 ABL생명보험 대표이사의 추천으로 이번 캠페인에 동참하게 됐다.,경제,1,val
9557,박신철 삼성SDI 품질보증실 프로가 세계에서 가장 많은 품질 자격증을 보유하게 됐다.,경제,1,val
15277,한은이 이번 금통위에서 금리를 인하할 것이라고 전망한 응답자는 1명에 그쳤다.,경제,1,val
9811,"한편, 한선화가 출연하는 OCN 구해줘2는 매주 수, 목요일 밤 11시에 방송된다.",연예,2,val


In [45]:
test_df = df.iloc[X_val]
train_df = df.iloc[X_train]

In [46]:
test_df.drop(columns='data_type', inplace=True)

/Users/datawhales/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [47]:
test_df.reset_index(drop=True, inplace=True)

test_df.head()

,sentence,topic,label
0,한편 캠핑클럽은 매주 일요일 오후 9시에 방송된다.,연예,2
1,장 대표는 시예저치앙 ABL생명보험 대표이사의 추천으로 이번 캠페인에 동참하게 됐다.,경제,1
2,박신철 삼성SDI 품질보증실 프로가 세계에서 가장 많은 품질 자격증을 보유하게 됐다.,경제,1
3,한은이 이번 금통위에서 금리를 인하할 것이라고 전망한 응답자는 1명에 그쳤다.,경제,1
4,"한편, 한선화가 출연하는 OCN 구해줘2는 매주 수, 목요일 밤 11시에 방송된다.",연예,2


In [48]:
train_df.drop(columns='data_type', inplace=True)

train_df.reset_index(drop=True, inplace=True)

train_df.head()

/Users/datawhales/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,sentence,topic,label
0,유전체 기반 오픈 플랫폼 마이지놈박스가 공식 웹사이트 오픈 4주년인 오는 8월 15...,IT/과학,5
1,해당 채널은 조 장관 딸 조모씨에 대한 영상을 올리며 실명과 얼굴을 그대로 노출하며...,IT/과학,5
2,뉴욕 증시는 그동안 무역합의가 이뤄질 것이란 데 대해 매우 낙관적이었다.,사회,0
3,이어 금번 WFP를 통해 지원되는 식량이 북한 주민에게 최대한 신속히 전달될 수 있...,사회,0
4,한국은행 통합별관 건축을 결국 계룡건설이 맡게 될 것으로 보인다.,경제,1


In [49]:
# split train_df into train_df, val_df
X_train, X_val, y_train, y_val = train_test_split(train_df.index.values,
                                                 train_df.label.values,
                                                 test_size=0.2,
                                                 random_state=42,
                                                 stratify=train_df.label.values)

In [50]:
val_df = train_df.iloc[X_val]
train_df = train_df.iloc[X_train]

In [54]:
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

In [55]:
train_df.head()

,sentence,topic,label
0,트리플하려고 버디를 많이 잡은 것 같다.,스포츠,8
1,'동상이몽2' 안현모가 '제 2차 북미정상회담' 뒷이야기를 공개한다.,연예,2
2,"그러면서 김혜준, 조이현, 김강훈 등 신예 배우들의 연기에도 집중해 줄 것을 당부했다.",문화,4
3,MBC 새 수목 드라마 신입사관 구해령 측은 4일 구해령을 비롯한 여사 4총사의 모...,연예,2
4,복지 확대를 통한 경제살리기의 구체적 방안으로 이 지사의 핵심 공약인 지역화폐를 통...,정치,6


In [56]:
val_df.head()

,sentence,topic,label
0,오는 23일로 예정된 중앙노동위원회 사후조정은 협상 타결로 가는 최대 분수령이 될 ...,경제,1
1,또한 업체 클라이언트를 위한 광고 이미지와 문구를 삽입할 수 있다.,생활,3
2,"한편 tvN 호텔 델루나 매주 토, 일 밤 9시에 방송된다.",생활,3
3,"배우 조윤희와 윤박, 오민석, 조우리가 주말 안방극장에 복귀한다.",연예,2
4,"이어이후 치료에 전념하며 2019년 3월 21일 재입대하였으나, 또다시 병증으로 인...",사회,0


In [57]:
test_df.head()

,sentence,topic,label
0,한편 캠핑클럽은 매주 일요일 오후 9시에 방송된다.,연예,2
1,장 대표는 시예저치앙 ABL생명보험 대표이사의 추천으로 이번 캠페인에 동참하게 됐다.,경제,1
2,박신철 삼성SDI 품질보증실 프로가 세계에서 가장 많은 품질 자격증을 보유하게 됐다.,경제,1
3,한은이 이번 금통위에서 금리를 인하할 것이라고 전망한 응답자는 1명에 그쳤다.,경제,1
4,"한편, 한선화가 출연하는 OCN 구해줘2는 매주 수, 목요일 밤 11시에 방송된다.",연예,2


In [58]:
print(len(train_df), len(val_df), len(test_df))

12460 3116 3894


In [59]:
label_dict

{'사회': 0,
 '경제': 1,
 '연예': 2,
 '생활': 3,
 '문화': 4,
 'IT/과학': 5,
 '정치': 6,
 '미용/건강': 7,
 '스포츠': 8}

In [60]:
import os

os.mkdir('../news_classification')

In [61]:
train_df.to_csv('../news_classification/news_cls_train.csv', index=False)
val_df.to_csv('../news_classification/news_cls_val.csv', index=False)
test_df.to_csv('../news_classification/news_cls_test.csv', index=False)